In [1]:
from sqlalchemy import create_engine, select, update
from sqlalchemy.orm import Session, joinedload
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import pickle as pk
import api.entities as db
from itemchm.hybrid_recommender import HybridRecommender
from itemchm.entities_repr import User, Book


In [2]:
# Create engine
ENGINE = create_engine("sqlite:///../../bookshelf.db", echo=True)

In [3]:
def sentiment_analysis(text : str):
    # Create a SentimentIntensityAnalyzer object
    sia = SentimentIntensityAnalyzer()

    # Perform sentiment analysis
    sentiment_scores = sia.polarity_scores(text)
    
    if sentiment_scores['compound'] < -0.1:
        return 1
    elif sentiment_scores['compound'] < 0.1:
        return 4
    else:
        return 6
        
def overall_rate(user_book : db.UserBook) -> float:
    RATING_WEIGHT = 4
    COMMENT_WEIGHT = 3
    SHARED_WEIGHT = 2
    READ_RATIO_WEIGHT = 1

    if user_book.rating == None:
        rating_value = 4
    else:
        rating_value = user_book.rating + 1
    
    if user_book.comment == None:
        comment_value = 4
    else:
        comment_value = sentiment_analysis(user_book.comment)
    
    read_ration_value = user_book.readRatio * 5 + 1

    if user_book.shared <= 0:
        shared_value = 1
    elif user_book.shared < 3:
        shared_value = 2
    elif user_book.shared < 6:
        shared_value = 3
    elif user_book.shared < 8:
        shared_value = 4
    elif user_book.shared < 12:
        shared_value = 5
    else:
        shared_value = 6

    return (RATING_WEIGHT * rating_value + COMMENT_WEIGHT * comment_value + SHARED_WEIGHT * shared_value + READ_RATIO_WEIGHT * read_ration_value) / (RATING_WEIGHT + COMMENT_WEIGHT + SHARED_WEIGHT + READ_RATIO_WEIGHT)

In [4]:
with Session(ENGINE) as session:
    db_users = session.query(db.User).all()
    db_books = session.query(db.Book).options(joinedload(db.Book.genres)).all()
    db_user_books = session.query(db.UserBook).all()

2024-08-27 21:04:51,775 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-27 21:04:51,780 INFO sqlalchemy.engine.Engine SELECT "Users".id AS "Users_id", "Users".name AS "Users_name", "Users".email AS "Users_email", "Users".features AS "Users_features" 
FROM "Users"
2024-08-27 21:04:51,781 INFO sqlalchemy.engine.Engine [generated in 0.00120s] ()
2024-08-27 21:04:51,800 INFO sqlalchemy.engine.Engine SELECT "Books".id AS "Books_id", "Books".name AS "Books_name", "Books".author AS "Books_author", "Books".language AS "Books_language", "Books".year AS "Books_year", "BookGenre_1"."booksId" AS "BookGenre_1_booksId", "BookGenre_1"."genresName" AS "BookGenre_1_genresName" 
FROM "Books" LEFT OUTER JOIN "BookGenre" AS "BookGenre_1" ON "Books".id = "BookGenre_1"."booksId"
2024-08-27 21:04:51,801 INFO sqlalchemy.engine.Engine [generated in 0.00118s] ()
2024-08-27 21:04:51,893 INFO sqlalchemy.engine.Engine SELECT "UserBooks".id AS "UserBooks_id", "UserBooks"."readRatio" AS "UserBooks_readRatio",

In [5]:
users_dict : dict[int, User] = {}

for db_user in db_users:
    user = User(db_user.id, db_user.name, {})
    users_dict[db_user.id] = user

books : list[Book] = []

for db_book in db_books:
    genres = [book_genre.genresName for book_genre in db_book.genres]
    book = Book(db_book.id, db_book.name, db_book.author, db_book.year, db_book.language, genres)
    books.append(book)

In [6]:
for user_book in db_user_books:
    rating = overall_rate(user_book)
    user = users_dict[user_book.userId]
    user.ratings[user_book.bookId] = rating

# Required Input
users = list(users_dict.values())

In [7]:
with open("books.pk", "wb") as f:
    books_pickle = pk.dump(books, f)
with open("users.pk", "wb") as u:
    users_pickle = pk.dump(users, u)

with open("books.pk", "rb") as f:
    books_pk = pk.load(f)

with open("users.pk", "rb") as u:
    users_pk = pk.load(u)


In [8]:
with open("books.pk", "rb") as f:
    books_pk = pk.load(f)

with open("users.pk", "rb") as u:
    users_pk = pk.load(u)

In [9]:
system = HybridRecommender(books_pk, users_pk)
system.compute_data()

Start Mixing Matrix
Start building Item Rating Similitud
Start building Item Rating Matrix
End building Item Rating Matrix
Start building Averages Book Array
End building Averages Book Array
End building Item Rating Similitud
Start building Group Rating Similitud
Start building Group Rating Matrix
Start 1 iteration of K-Means
End 1 iteration of K-Means
Start 2 iteration of K-Means
End 2 iteration of K-Means
Start 3 iteration of K-Means
End 3 iteration of K-Means
Start 4 iteration of K-Means
End 4 iteration of K-Means
Start 5 iteration of K-Means
End 5 iteration of K-Means
Start 6 iteration of K-Means
End 6 iteration of K-Means
Start 7 iteration of K-Means
End 7 iteration of K-Means
Start 8 iteration of K-Means
End 8 iteration of K-Means
Start 9 iteration of K-Means
End 9 iteration of K-Means
Start 10 iteration of K-Means
End 10 iteration of K-Means
End building Group Rating Matrix
Start building Averages User Array
End building Averages User Array
End building Group Rating Similitud
En

In [10]:
with open("system.pk", "wb") as s:
    sy = pk.dump(system, s)